In [1]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf
sys.path.append('..')
from tfomics import neuralnetwork_dev as nn
from tfomics import neuralbuild as nb
from tfomics import utils, learn

# import models
from model_zoo import test_model

In [2]:
filename = 'processed_dataset.hdf5'
#data_path = 'D:/Dropbox/TFconnect'
data_path = '/home/peter/Code/tensorflow/data'
filepath = os.path.join(data_path,filename)

group_name = ['processed_data']
dataset = h5py.File(filepath,'r')
%time dtf = np.array(dataset['/'+group_name[0]+'/dtf'])
ltf = np.array(dataset['/'+group_name[0]+'/ltf'])
dtf_crossval = np.array(dataset['/'+group_name[0]+'/dtf_crossval'])
ltf_crossval = np.array(dataset['/'+group_name[0]+'/ltf_crossval'])

X_train = dtf#[:10000,:,:,:]
y_train = ltf#[:10000,:]
X_valid = dtf_crossval#[:5000,:,:,:]
y_valid = ltf_crossval#[:5000,:]

CPU times: user 468 ms, sys: 976 ms, total: 1.44 s
Wall time: 1.44 s


In [3]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  


# create model
layer1 = {'layer': 'input',
        'input_shape': input_shape,
        'name': 'input'
        }
layer2 = {'layer': 'conv2d', 
        'num_filters': 18,
        'filter_size': (2, 5),
        'norm': 'batch',
        'activation': 'leaky_relu',
        'name': 'conv1'
        }
layer3 = {'layer': 'conv2d', 
        'num_filters': 40,
        'filter_size': (2, 5),
        'norm': 'batch',
        'activation': 'leaky_relu',
        'dropout': 0.1,
        'pool_size': (1,10),
        'name': 'conv2'
        }
layer4 = {'layer': 'conv2d', 
        'num_filters': 15,
        'filter_size': (1,1),
        'norm': 'batch',
        'activation': 'leaky_relu',
        'name': 'conv3'
        }
layer5 = {'layer': 'dense', 
        'num_units': 100,
        'activation': 'leaky_relu',
        'dropout': 0.5,
        'name': 'dense1'
        }
layer6 = {'layer': 'dense', 
        'num_units': num_labels,
        'activation': 'softmax',
        'name': 'dense2'
        }

#from tfomics import build_network
model_layers = [layer1, layer2, layer3, layer4, layer5, layer6]

# optimization parameters
optimization = {"objective": "categorical",
              "optimizer": "adam",
              "learning_rate": 0.001,      
              "l2": 1e-6,
              # "l1": 0, 
              }

In [4]:
nnmodel.network['conv3_batch'].decay

NameError: name 'nnmodel' is not defined

In [5]:
nnbuild = nb.NeuralBuild(model_layers)
network, placeholders, hidden_feed_dict = nnbuild.get_network_build()

# build neural network class
nnmodel = nn.NeuralNet(network, placeholders, hidden_feed_dict)
nnmodel.inspect_layers()

# set output file paths

results_path = utils.make_directory(data_path, 'results')
results_path = utils.make_directory(results_path, 'tfomics')
output_name = 'test'
filepath = os.path.join(results_path, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: input
(?, 3, 330, 1)
layer2: conv1
(?, 2, 326, 18)
layer3: conv1_batch
(?, 2, 326, 18)
layer4: conv1_active
(?, 2, 326, 18)
layer5: conv2
(?, 1, 322, 40)
layer6: conv2_batch
(?, 1, 322, 40)
layer7: conv2_active
(?, 1, 322, 40)
layer8: conv2_pool
(?, 1, 32, 40)
layer9: conv2_dropout
(?, 1, 32, 40)
layer10: conv3
(?, 1, 32, 15)
layer11: conv3_batch
(?, 1, 32, 15)
layer12: conv3_active
(?, 1, 32, 15)
layer13: dense1
(?, 100)
layer14: dense1_bias
(?, 100)
layer15: dense1_active
(?, 100)
layer16: dense1_dropout
(?, 100)
layer17: dense2
(?, 2)
layer18: dense2_bias
(?, 2)
layer19: dense2_active
(?, 2)
layer20: output
(?, 2)
----------------------------------------------------------------------------


In [6]:
train = {'inputs': [X_train], 'targets': y_train}
valid = {'inputs': [X_valid], 'targets': y_valid}
data = {'train': train, 'valid': valid}
learn.train_minibatch(nntrainer, data, batch_size=1000, num_epochs=50, 
                    patience=10, verbose=2, shuffle=True)

Epoch 1 out of 50 
{<tf.Tensor 'input:0' shape=(?, 3, 330, 1) dtype=float32>: array([[[[-0.73375756],
         [ 0.06282479],
         [ 1.66912007],
         ..., 
         [ 0.31230387],
         [ 1.40651047],
         [ 1.51593101]],

        [[-0.53403217],
         [ 0.41173071],
         [ 1.74936318],
         ..., 
         [-0.90706205],
         [-0.70735919],
         [-0.729967  ]],

        [[-0.38247883],
         [ 0.28714275],
         [ 1.9718976 ],
         ..., 
         [-0.74859875],
         [-0.58545607],
         [-0.50883335]]],


       [[[ 1.00188351],
         [ 0.75285172],
         [ 1.07244229],
         ..., 
         [-0.94056427],
         [ 0.34194928],
         [ 1.74897885]],

        [[ 0.85650045],
         [ 0.92813373],
         [ 1.21864665],
         ..., 
         [-1.28453922],
         [ 0.06853413],
         [ 1.53701675]],

        [[-0.778054  ],
         [-0.67638952],
         [-0.50116462],
         ..., 
         [-0.60249001],
    

FailedPreconditionError: Attempting to use uninitialized value cond/moments/normalize/mean/ExponentialMovingAverage/biased
	 [[Node: cond/moments/normalize/mean/ExponentialMovingAverage/biased/read = Identity[T=DT_FLOAT, _class=["loc:@cond/moments/normalize/mean/ExponentialMovingAverage"], _device="/job:localhost/replica:0/task:0/gpu:0"](cond/moments/normalize/mean/ExponentialMovingAverage/biased)]]
	 [[Node: moments_2/sufficient_statistics/Shape/_31 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_1873_moments_2/sufficient_statistics/Shape", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'cond/moments/normalize/mean/ExponentialMovingAverage/biased/read', defined at:
  File "/home/peter/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/pet